In [1]:
import json
import pandas as pd

from pathlib import Path

In [2]:
current_folder = Path().resolve()

In [3]:
def manipulate_raw_data(l):
    recipe_v1 = [x.strip() for x in l]
    
    # cut top and bottom lines
    extract_v1 = recipe_v1[recipe_v1.index('topBanner') + 1:recipe_v1.index("You might also like...")]
    
    # find check items to extract relevant information
    ident = []
    check = ["Preparation Time","Ingredients","Nutritional information","Boxes and ingredients","Instructions"]
    for idx, val in enumerate(extract_v1):
        if 'Tag' in val:
            continue
        for item in check:
            if item in val:
                ident.append(idx)       

    title = " ".join(extract_v1[5:7])
    prep_time = extract_v1[ident[0]].split("Time")[1]
    
    ingred = extract_v1[ident[1]+3:ident[2]]    
    ingredi = [" ".join(ingred[idx:idx+3:2]) for idx in range(len(ingred)) if len(ingred[idx]) > 0 and ingred[idx][0].isnumeric()]

    nutri = extract_v1[ident[2]:ident[3]]

    instr = extract_v1[ident[4]:]
    instr = " ".join([line for line in instr if not (line.isnumeric() or line == "PDF" or line == "Instructions")])
    
    return title, prep_time, ingredi, nutri, instr

In [4]:
recipes = {}

path_folder = current_folder.parents[0] / 'data' / 'predict_cooking_time' / 'recipes_raw' 

# find number of recipes
path_glob_generator = Path(path_folder).glob('**/*')
files = [x for x in path_glob_generator if x.is_file()]

In [5]:
for x in range(1,len(files) + 1):
    file = "".join([r"R", str(x), ".txt"])

    with open(path_folder / file, encoding = 'utf-8') as f:
        recipe = f.readlines()

    title, prep_time, ingredi, nutri, instr_r = manipulate_raw_data(recipe)

    recipe = {}
    recipe['prep_time'] = prep_time
    recipe['ingredients'] = ingredi
    recipe['nutritional'] = nutri
    recipe['instructions'] = instr_r
    recipe['ordered'] = 1
    
    if title in recipes.keys():
        recipes[title]['ordered'] += 1
    else:
        recipes[title] = recipe

In [6]:
# Serializing json 
json_object = json.dumps(recipes, indent=4, sort_keys=True)

output_path = current_folder.parents[0] / 'data' / 'predict_cooking_time' / 'processed_data' / 'recipes.json'
  
# Writing to sample.json
with open(output_path, "w", encoding = 'utf-8') as outfile:
    outfile.write(json_object)

In [7]:
# JSON file
f = open(output_path, "r")
  
# Reading from file
data = json.loads(f.read())